#  Setup stuff: Connectivity

In [1]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [2]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [9]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [3]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


EK9giUcZHxAMni4XK77q35j5EHyoQGW6NbfWuzosDCrW


In [10]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=2)

print(my_graph)


<_Graph my_graph, yMo6MeZejRfDVrxnCGU4bokorQsMZt5kgFinm4RLKBa, 0>


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

# Create the vertices/nodes, edges ..

In [5]:
import pandas as pd

print("--")


--


In [6]:

df_airports = pd.DataFrame([
   ["MKE", "Milwaukee"                     ],
   ["ORD", "Chicago O-Hare"                ],
   ["SJC", "San Jose"                      ],
   ["DEN", "Denver"                        ] ,
   ], columns = ['airport_code', 'airport_name'])

df_airports.head(10)


,airport_code,airport_name
0,MKE,Milwaukee
1,ORD,Chicago O-Hare
2,SJC,San Jose
3,DEN,Denver


In [11]:
#  Create nodes, airports

import contextlib

for l_each in df_airports.iterrows():
    
   l_airport_code = l_each[1][0]
   l_airport_name = l_each[1][1]
    
   l_query = """

      //  Was using ..
      //
      // CREATE ( n: Airport {{ airport_code: '{0}' }} ) 
      // SET n.airport_name = '{1}' 
      
      //  Was told to use ..
      MERGE ( n: Airport {{ airport_code: '{0}' }})
      ON CREATE SET n.airport_name = '{1}'
   
      """.format(l_airport_code, l_airport_name)
         
    
   l_result0 = my_graph.query(l_query)
        
   print(".", end='')


print("  ")
print("--")


          0/? [?op/s]

.

          0/? [?op/s]

.

          0/? [?op/s]

.

          0/? [?op/s]

.  
--


In [12]:
#  What's in the vertices/nodes ..

l_result = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)
print(l_result[0:10])

#  When using MERGE code above
#
#     n.internal_id   n.labels n.airport_code n.type n.airport_name
#  0              0  [Airport]            SJC   node            NaN
#  1              1  [Airport]            MKE   node      Milwaukee
#  2              2  [Airport]            ORD   node       San Jose
#  3              3  [Airport]            DEN   node         Denver

#  When using CREATE code above
#
#     n.internal_id   n.labels n.airport_code  n.airport_name n.type
#  2              0  [Airport]            MKE       Milwaukee   node
#  1              1  [Airport]            ORD  Chicago O-Hare   node
#  0              2  [Airport]            SJC        San Jose   node
#  3              3  [Airport]            DEN          Denver   node




          0/? [?op/s]

     n.internal_id   n.labels n.airport_code  n.airport_name n.type
0                0  [Airport]            SJC        San Jose   node
1  281474976710656  [Airport]            MKE       Milwaukee   node
2  281474976710657  [Airport]            ORD  Chicago O-Hare   node
3  281474976710658  [Airport]            DEN          Denver   node


In [ ]:
#  Delete all edges, nodes for test reset

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   RETURN count(r)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n)
   RETURN count(n)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   DELETE r
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n)
   DELETE n
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   RETURN count(r)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n)
   RETURN count(n)
   """)
display(print(l_result))


In [ ]:
# ##################################################################

In [9]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
   ["MKE", "ORD",   66],
   ["ORD", "MKE",   66],
      #
   ["ORD", "DEN",  886],
   ["DEN", "ORD",  886],
      #
   ["SJC", "DEN",  948],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948],
      #
   ["SJC", "ORD", 1829],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ['START_ID', 'END_ID', 'LENGTH'])

df_flights.head(10)


,START_ID,END_ID,LENGTH
0,MKE,ORD,66
1,ORD,MKE,66
2,ORD,DEN,886
3,DEN,ORD,886
4,SJC,DEN,948
5,DEN,SJC,948
6,SJC,ORD,1829


In [10]:

for l_each in df_flights.iterrows():
    
   l_startid = l_each[1][0]
   l_endid   = l_each[1][1]
   l_length  = l_each[1][2]
    
   l_query = """

      MATCH (n: Airport ) WHERE n.airport_code = '{0}'
      MATCH (m: Airport ) WHERE m.airport_code = '{1}'
      CREATE (n) -[r:FLIES_TO {{ LENGTH: {2} }}]-> (m)
      
      """.format(l_startid, l_endid, l_length)
    
   l_result0 = my_graph.query(l_query)
        
   print(".", end='')


print("  ")
print("--")


          0/? [?op/s]

.

          0/? [?op/s]

.

          0/? [?op/s]

...

.

          0/? [?op/s]

.

          0/? [?op/s]

.  
--


In [ ]:
#  What's in the edge .. 

l_result = my_graph.query("""
   MATCH ( n )  - [r] ->  (m)
   RETURN n.airport_code AS FROM, m.airport_code AS TO, r.LENGTH AS LENGTH
   """)
print(l_result[0:10])


####  Finally now, the Analytics ..

In [ ]:
#  First traversal, Airport --> Airport, up to 3 in distance ..

l_result = my_graph.query("""

   MATCH (n: Airport )  - [r: FLIES_TO *..3] ->  (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code, m.airport_code
   
   """)

print(l_result[0:10])

#    m.airport_code n.airport_code
#  0            MKE            SJC
#  1            MKE            SJC

#  So Yes, answer, but no path ..



In [ ]:
#  Adding path ..

l_result = my_graph.query("""

   MATCH p= ( n: Airport )  - [r: FLIES_TO *..3] ->  (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code AS FROM, m.airport_code AS TO, p
   """)

print(l_result[0:10])

#  Currently, error ..
#
#  OperationError: query 9tWneccAey6jzpKLGGtto7v3X1iSkQDdRrSa7gcuv8fD-3qJysJrdkEo1NWz7V error: backtrace
#  (QueryClient.cpp:456): opgraph check failed (OpGraph.cpp:429): Unimplemented Features Pass
#  (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:360): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:658): backtrace (UnimplementedFeaturesChecker.cpp:214):
#  accessing path variable p is not supported (UnimplementedFeaturesChecker.cpp:24): InternalError:NotImplemented



In [ ]:
#  Using COLLECT to try and get path ..

l_result = my_graph.query("""

   MATCH ( n: Airport )  - [r: FLIES_TO *..3] ->  (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code AS FROM, COLLECT(m.airport_code)
   """)

print(l_result[0:10])

#    COLLECT(m.airport_code) FROM
#  0              [MKE, MKE]  SJC

#  Not what we want



In [ ]:
#  Another attempt at using COLLECT 

l_result = my_graph.query("""

   MATCH ( n: Airport )  - [r: FLIES_TO *..3] ->  (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code AS FROM, COLLECT(r.END_ID)
   """)

print(l_result[0:10])


#  OperationError: query 6jxiUgnCFao3yf2BTXPJqQDUTkAwTL4LMzmKjF6kNmD4-4BeDpVqisXe9Ha8QW error:
#  backtrace (QueryClient.cpp:452): backtrace (OpGraph.cpp:313): backtrace (OpGraph.h:588):
#  backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace
#  (OpGraph.cpp:320): backtrace (RewritePattern.h:46): backtrace (MatchOrderOptimizer.cpp:102):
#  backtrace (MatchOperator.cpp:2062): returning named variable paths not yet supported
#  (MatchOperator.cpp:1894): InternalError:NotImplemented



In [ ]:
# ##################################################################

In [ ]:
#  Second traversal, Airport --> Airport, up to 3 in distance, but in other direction ..

l_result = my_graph.query("""

   MATCH (n: Airport )  - [r: FLIES_TO *..3] ->  (m: Airport)
   WHERE n.airport_code = "MKE" AND m.airport_code = "SJC"
   RETURN n.airport_code, m.airport_code
   
   """)

print(l_result[0:10])

In [ ]:
# ##################################################################

In [ ]:
#  https://stackoverflow.com/questions/27600608/traverse-path-cypher-query